# Load JSON and Metadata filter JSON field or array

In this notebook, we are going to use Kaggle IMDB data, available as either raw JSON or CSV. We'll load it into Milvus vector database, then search with a metadata filter.

Let's get started!

In [1]:
# For colab install these libraries in this order:
# !pip install numpy pandas torch pymilvus langchain transformers sentence-transformers 
# !pip install python-dotenv

In [2]:
# Import common libraries.
import sys, os, time, pprint, json
import numpy as np
import pandas as pd

# Import custom functions for splitting and search.
sys.path.append("..")  # Adds higher directory to python modules path.
import milvus_utilities as _utils

# Decide if you want to read JSON or CSV.
DATA_TYPE = "JSON" # "CSV" or "JSON"

## Read JSON data into a pandas dataframe

The JSON data comes from https://www.kaggle.com/datasets/nelepie/imdb-genre-classification

In [3]:
if DATA_TYPE == "JSON":

    # Read some JSON data.
    df = pd.read_json('data/parsed_data.json')
    print(f"df shape: {df.shape}")
    df = df.T

    # Save top 100 rows as tiny json file
    temp = df.head(100).copy()
    temp.to_json('data/tiny_parsed_data.json')
    df = pd.read_json('data/tiny_parsed_data.json')

    # Reset index and call it movie_index.
    df = df.reset_index().rename(columns={'index': 'movie_index'})

    # Drop release date, it has too many nulls.
    df.drop(columns=['releaseDate'], inplace=True)

    # Reverse column names label, genres.
    df = df.rename(columns={'labels': 'Genres'})
    df = df.rename(columns={'genre': 'labels'})

    # Convert year to a number.
    df['film_year'] = df.film_year.astype(int)

    # Concatenate Title and Description into 'text' column.
    df['text'] = df['title'] + ' ' + df['description']

    # Verify data is valid json.
    try:
        # Convert temp to a JSON string and try to parse it
        json.loads(df.to_json())
        print("data is valid JSON.")
    except json.JSONDecodeError:
        print("data is not valid JSON.")

    print(f"df shape: {df.shape}")
    print(df.dtypes)
    display(df.head(2))

    # Inspect text.
    print(f"Example text length: {len(df.text[0])}")
    print(f"Example text: {df.text[0]}")

df shape: (7, 10357)
data is valid JSON.
df shape: (100, 8)
movie_index    object
title          object
description    object
poster_url     object
labels         object
Genres         object
film_year       int64
text           object
dtype: object


,movie_index,title,description,poster_url,labels,Genres,film_year,text
0,tt6443346,Black Adam,"Nearly 5,000 years after he was bestowed with ...",https://m.media-amazon.com/images/M/MV5BYzZkOG...,SuperHero,"[Action, Adventure, Fantasy]",2022,"Black Adam Nearly 5,000 years after he was bes..."
1,tt10954600,Ant-Man and the Wasp: Quantumania,"Scott Lang and Hope Van Dyne, along with Hank ...",https://m.media-amazon.com/images/M/MV5BNDgyNG...,SuperHero,"[Action, Adventure, Comedy]",2023,Ant-Man and the Wasp: Quantumania Scott Lang a...


Example text length: 240
Example text: Black Adam Nearly 5,000 years after he was bestowed with the almighty powers of the Egyptian gods - and imprisoned just as quickly - Black Adam is freed from his earthly tomb, ready to unleash his unique form of justice on the modern world.


## Read CSV data into a pandas dataframe

The data used in this notebook is [Kaggle 48K movies](https://www.kaggle.com/datasets/yashgupta24/48000-movies-dataset) which contains a lot of metadata in addition to the raw review text.

Usually there is a data cleaning step.  Such as replace empty strings with "" or unusual and empty fields with median values.  Below, I'll just drop rows with null values.

In [4]:
if DATA_TYPE == "CSV":

    # Read CSV data.
    df = pd.read_csv("data/kaggle_imdb_small.csv")

    # Concatenate 'Name', 'Keywords', and 'Description' into 'text' column
    df['text'] = df['Name'] + ' ' + df['Description'] + ' ' + df['ReviewBody']

    # Convert genres from string with commas in it to list of strings.
    df['Genres'] = df['Genres'].str.split(',')

    # Convert actors from string with commas in it to list of strings.
    df['Actors'] = df['Actors'].str.split(',')

    # Convert keywords from string with commas in it to list of strings.
    df['Keywords'] = df['Keywords'].str.split(',')

    # Extract out just the year from the date.
    df['DatePublished'] = df.DatePublished.apply(lambda x: x.split('-')[0])
    df['DatePublished'] = df.DatePublished.astype(int)

    # Drop extra rating columns.
    df.drop(columns=['RatingCount', 'BestRating', 'WorstRating'], inplace=True)

    # Inspect text.
    print(f"Example text length: {len(df.text[0])}")
    pprint.pprint(f"Example text: {df.text[0]}")

    print(df.dtypes)
    display(df.head(2))

## Start up a Zilliz free tier cluster.

Code in this notebook uses fully-managed Milvus on [Ziliz Cloud free trial](https://cloud.zilliz.com/login).  
  1. Choose the default "Starter" option when you provision > Create collection > Give it a name > Create cluster and collection.  
  2. On the Cluster main page, copy your `API Key` and store it locally in a .env variable.  See note below how to do that.
  3. Also on the Cluster main page, copy the `Public Endpoint URI`.

💡 Note: To keep your tokens private, best practice is to use an **env variable**.  See [how to save api key in env variable](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety). <br>

👉🏼 In Jupyter, you need a .env file (in same dir as notebooks) containing lines like this:
- ZILLIZ_API_KEY=f370c...
- OPENAI_API_KEY=sk-H...
- VARIABLE_NAME=value...

In [5]:
# STEP 1. CONNECT TO ZILLIZ CLOUD

# !pip install pymilvus #python sdk for milvus
from pymilvus import connections, utility
from dotenv import load_dotenv
load_dotenv()
TOKEN = os.getenv("ZILLIZ_API_KEY")

# Connect to Zilliz cloud using endpoint URI and API key TOKEN.
# TODO change this.
CLUSTER_ENDPOINT="https://in03-xxxx.api.gcp-us-west1.zillizcloud.com:443"
CLUSTER_ENDPOINT="https://in03-48a5b11fae525c9.api.gcp-us-west1.zillizcloud.com:443"
connections.connect(
  alias='default',
  #  Public endpoint obtained from Zilliz Cloud
  uri=CLUSTER_ENDPOINT,
  # API key or a colon-separated cluster username and password
  token=TOKEN,
)

# Check if the server is ready and get collection name.
print(f"Type of server: {utility.get_server_version()}")

Type of server: Zilliz Cloud Vector Database(Compatible with Milvus 2.3)


## Load the Embedding Model checkpoint and use it to create vector embeddings
**Embedding model:**  We will use the open-source [sentence transformers](https://www.sbert.net/docs/pretrained_models.html) available on HuggingFace to encode the documentation text.  We will download the model from HuggingFace and run it locally. 

💡Tip:  A good way to choose a sentence transformer model is to check the [MTEB Leaderboard](https://huggingface.co/spaces/mteb/leaderboard).  Sort descending by column "Retrieval Average" and choose the best-performing small model.

Two model parameters of note below:
1. EMBEDDING_DIM refers to the dimensionality or length of the embedding vector. In this case, the embeddings generated for EACH token in the input text will have the SAME length = 1024. This size of embedding is often associated with BERT-based models, where the embeddings are used for downstream tasks such as classification, question answering, or text generation. <br><br>
2. MAX_SEQ_LENGTH is the maximum Context Length the encoder model can handle for input sequences. In this case, if sequences longer than 512 tokens are given to the model, everything longer will be (silently!) chopped off.  This is the reason why a chunking strategy is needed to segment input texts into chunks with lengths that will fit in the model's input.

In [6]:
# STEP 2. DOWNLOAD AN OPEN SOURCE EMBEDDING MODEL.

# Import torch.
import torch
from sentence_transformers import SentenceTransformer

# Initialize torch settings
torch.backends.cudnn.deterministic = True
DEVICE = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print(f"device: {DEVICE}")

# Load the model from huggingface model hub.
model_name = "WhereIsAI/UAE-Large-V1"
encoder = SentenceTransformer(model_name, device=DEVICE)
print(type(encoder))
print(encoder)

# Get the model parameters and save for later.
EMBEDDING_DIM = encoder.get_sentence_embedding_dimension()
MAX_SEQ_LENGTH_IN_TOKENS = encoder.get_max_seq_length() 
# # Assume tokens are 3 characters long.
# MAX_SEQ_LENGTH = MAX_SEQ_LENGTH_IN_TOKENS * 3
# HF_EOS_TOKEN_LENGTH = 1 * 3
# Test with 512 sequence length.
MAX_SEQ_LENGTH = MAX_SEQ_LENGTH_IN_TOKENS
HF_EOS_TOKEN_LENGTH = 1

# Inspect model parameters.
print(f"model_name: {model_name}")
print(f"EMBEDDING_DIM: {EMBEDDING_DIM}")
print(f"MAX_SEQ_LENGTH: {MAX_SEQ_LENGTH}")

device: cpu
<class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)
model_name: WhereIsAI/UAE-Large-V1
EMBEDDING_DIM: 1024
MAX_SEQ_LENGTH: 512


## Create a Milvus collection

You can think of a collection in Milvus like a "table" in SQL databases.  The **collection** will contain the 
- **Schema** (or [no-schema Milvus client](https://milvus.io/docs/using_milvusclient.md)).  
💡 You'll need the vector `EMBEDDING_DIM` parameter from your embedding model.
Typical values are:
   - 1024 for sbert embedding models
   - 1536 for ada-002 OpenAI embedding models
- **Vector index** for efficient vector search
- **Vector distance metric** for measuring nearest neighbor vectors
- **Consistency level**
In Milvus, transactional consistency is possible; however, according to the [CAP theorem](https://en.wikipedia.org/wiki/CAP_theorem), some latency must be sacrificed. 💡 Searching movie reviews is not mission-critical, so [`eventually`](https://milvus.io/docs/consistency.md) consistent is fine here.

## Add a Vector Index

The vector index determines the vector **search algorithm** used to find the closest vectors in your data to the query a user submits.  

Most vector indexes use different sets of parameters depending on whether the database is:
- **inserting vectors** (creation mode) - vs - 
- **searching vectors** (search mode) 

Scroll down the [docs page](https://milvus.io/docs/index.md) to see a table listing different vector indexes available on Milvus.  For example:
- FLAT - deterministic exhaustive search
- IVF_FLAT or IVF_SQ8 - Hash index (stochastic approximate search)
- HNSW - Graph index (stochastic approximate search)
- AUTOINDEX - Automatically determined based on OSS vs [Zilliz cloud](https://docs.zilliz.com/docs/autoindex-explained), type of GPU, size of data.

Besides a search algorithm, we also need to specify a **distance metric**, that is, a definition of what is considered "close" in vector space.  In the cell below, the [`HNSW`](https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md) search index is chosen.  Its possible distance metrics are one of:
- L2 - L2-norm
- IP - Dot-product
- COSINE - Angular distance

💡 Most use cases work better with normalized embeddings, in which case L2 is useless (every vector has length=1) and IP and COSINE are the same.  Only choose L2 if you plan to keep your embeddings unnormalized.

In [7]:
# STEP 3. CREATE A NO-SCHEMA MILVUS COLLECTION AND DEFINE THE DATABASE INDEX.

from pymilvus import MilvusClient

# Set the Milvus collection name.
COLLECTION_NAME = "imdb_metadata"

# Add custom HNSW search index to the collection.
# M = max number graph connections per layer. Large M = denser graph.
# Choice of M: 4~64, larger M for larger data and larger embedding lengths.
M = 16
# efConstruction = num_candidate_nearest_neighbors per layer. 
# Use Rule of thumb: int. 8~512, efConstruction = M * 2.
efConstruction = M * 2
# Create the search index for local Milvus server.
INDEX_PARAMS = dict({
    'M': M,               
    "efConstruction": efConstruction })
index_params = {
    "index_type": "HNSW", 
    "metric_type": "COSINE", 
    "params": INDEX_PARAMS
    }

# Use no-schema Milvus client uses flexible json key:value format.
# https://milvus.io/docs/using_milvusclient.md
mc = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    # API key or a colon-separated cluster username and password
    token=TOKEN)

# Check if collection already exists, if so drop it.
has = utility.has_collection(COLLECTION_NAME)
if has:
    drop_result = utility.drop_collection(COLLECTION_NAME)
    print(f"Successfully dropped collection: `{COLLECTION_NAME}`")

# Create the collection.
mc.create_collection(COLLECTION_NAME, 
                     EMBEDDING_DIM,
                     consistency_level="Eventually", 
                     auto_id=True,  
                     overwrite=True,
                     # skip setting params below, if using AUTOINDEX
                     params=index_params
                    )

print(f"Successfully created collection: `{COLLECTION_NAME}`")
pprint.pprint(mc.describe_collection(COLLECTION_NAME))

Successfully created collection: `imdb_metadata`
{'aliases': [],
 'auto_id': True,
 'collection_id': 448076879578126663,
 'collection_name': 'imdb_metadata',
 'consistency_level': 3,
 'description': '',
 'enable_dynamic_field': True,
 'fields': [{'auto_id': True,
             'description': '',
             'field_id': 100,
             'is_primary': True,
             'name': 'id',
             'params': {},
             'type': <DataType.INT64: 5>},
            {'description': '',
             'field_id': 101,
             'name': 'vector',
             'params': {'dim': 1024},
             'type': <DataType.FLOAT_VECTOR: 101>}],
 'num_partitions': 1,
 'num_shards': 1,
 'properties': {}}


## Chunking

Before embedding, it is necessary to decide your chunk strategy, chunk size, and chunk overlap.  In this demo, I will use:
- **Strategy** = Keep movie reveiws as single chunks unless they are too long.
- **Chunk size** = Use the embedding model's parameter `MAX_SEQ_LENGTH`
- **Overlap** = Rule-of-thumb 10-15%
- **Function** = Langchain's convenient `RecursiveCharacterTextSplitter` to split up long reviews recursively.

In [8]:
# STEP 4. PREPARE DATA: CHUNK AND EMBED
from langchain.text_splitter import RecursiveCharacterTextSplitter

def recursive_splitter_wrapper(text, chunk_size):

    # Default chunk overlap is 10% chunk_size.
    chunk_overlap = np.round(chunk_size * 0.10, 0)

    # Use langchain's convenient recursive chunking method.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    chunks: List[str] = text_splitter.split_text(text)

    # Replace special characters with spaces.
    chunks = [text.replace("<br /><br />", " ") for text in chunks]

    return chunks

# Use recursive splitter to chunk text.
def imdb_chunk_text(batch_size, df, chunk_size):

    batch = df.head(batch_size).copy()
    print(f"chunk size: {chunk_size}")
    print(f"original shape: {batch.shape}")
    
    start_time = time.time()

    # 1. Chunk the text review into chunk_size.
    batch['chunk'] = batch['text'].apply(recursive_splitter_wrapper, chunk_size=chunk_size)
    # Explode the 'chunk' column to create new rows for each chunk.
    batch = batch.explode('chunk', ignore_index=True)
    print(f"new shape: {batch.shape}")

    # 2. Add embeddings as new column in df.
    embeddings = torch.tensor(encoder.encode(batch['chunk']))
    # Normalize the embeddings.
    embeddings = np.array(embeddings / np.linalg.norm(embeddings))

    # 3. Convert embeddings to list of `numpy.ndarray`, each containing `numpy.float32` numbers.
    converted_values = list(map(np.float32, embeddings))
    batch['vector'] = converted_values

    end_time = time.time()
    print(f"Chunking + embedding time for {batch_size} docs: {end_time - start_time} sec")
    # Inspect the batch of data.
    assert len(batch.chunk[0]) <= MAX_SEQ_LENGTH-1
    assert len(batch.vector[0]) == EMBEDDING_DIM
    print(f"type embeddings: {type(batch.vector)} of {type(batch.vector[0])}")
    print(f"of numbers: {type(batch.vector[0][0])}")

    return batch

In [9]:
## Chunk and Embed Text Data

# Use the embedding model parameters.
# chunk_size = MAX_SEQ_LENGTH - HF_EOS_TOKEN_LENGTH
chunk_size = 512
chunk_overlap = np.round(chunk_size * 0.10, 0)

# Chunk a batch of data from pandas DataFrame and inspect it.
BATCH_SIZE = 100
batch = imdb_chunk_text(BATCH_SIZE, df, chunk_size)
display(batch.head(2))

# Drop the original text column, keep the new 'chunk' column.
batch.drop(columns=['text'], axis=1, inplace=True)

chunk size: 512
original shape: (100, 8)
new shape: (100, 9)
Chunking + embedding time for 100 docs: 6.592205047607422 sec
type embeddings: <class 'pandas.core.series.Series'> of <class 'numpy.ndarray'>
of numbers: <class 'numpy.float32'>


,movie_index,title,description,poster_url,labels,Genres,film_year,text,chunk,vector
0,tt6443346,Black Adam,"Nearly 5,000 years after he was bestowed with ...",https://m.media-amazon.com/images/M/MV5BYzZkOG...,SuperHero,"[Action, Adventure, Fantasy]",2022,"Black Adam Nearly 5,000 years after he was bes...","Black Adam Nearly 5,000 years after he was bes...","[0.0040127905, 0.001754614, -0.0010465175, 0.0..."
1,tt10954600,Ant-Man and the Wasp: Quantumania,"Scott Lang and Hope Van Dyne, along with Hank ...",https://m.media-amazon.com/images/M/MV5BNDgyNG...,SuperHero,"[Action, Adventure, Comedy]",2023,Ant-Man and the Wasp: Quantumania Scott Lang a...,Ant-Man and the Wasp: Quantumania Scott Lang a...,"[0.008013694, -0.001968402, 0.0013089634, 0.00..."


## Insert data into Milvus

For each original text chunk, we'll write the quadruplet (`vector, text, source, h1, h2`) into the database.

<div>
<img src="../../images/db_insert.png" width="80%"/>
</div>

**The Milvus Client wrapper can only handle loading data from a list of dictionaries.**

Otherwise, in general, Milvus supports loading data from:
- pandas dataframes 
- list of dictionaries

Below, we use the embedding model provided by HuggingFace, download its checkpoint, and run it locally as the encoder.  

In [10]:
# STEP 5. INSERT CHUNKS AND EMBEDDINGS IN ZILLIZ.

# Convert the DataFrame to a list of dictionaries
chunk_list = batch.to_dict(orient='records')

# Insert data into the Milvus collection.
print("Start inserting entities")
start_time = time.time()
insert_result = mc.insert(
    COLLECTION_NAME,
    data=chunk_list,
    progress_bar=True)
end_time = time.time()
print(f"Milvus Client insert time for {batch.shape[0]} vectors: {end_time - start_time} seconds")

# Milvus Client does an automatic flush, save data to S3.

# CSV: Milvus Client insert time for 456 vectors: 8.96166205406189 seconds
# JSON: Milvus Client insert time for 100 vectors: 0.3610990047454834 seconds

Start inserting entities
Milvus Client insert time for 100 vectors: 0.36271023750305176 seconds


In [11]:
# TODO - Uncomment to print a single row.
chunk_list = batch.to_dict(orient='records')
chunk_list[0]

{'movie_index': 'tt6443346',
 'title': 'Black Adam',
 'description': 'Nearly 5,000 years after he was bestowed with the almighty powers of the Egyptian gods - and imprisoned just as quickly - Black Adam is freed from his earthly tomb, ready to unleash his unique form of justice on the modern world.',
 'poster_url': 'https://m.media-amazon.com/images/M/MV5BYzZkOGUwMzMtMTgyNS00YjFlLTg5NzYtZTE3Y2E5YTA5NWIyXkEyXkFqcGdeQXVyMjkwOTAyMDU@._V1_QL75_UX190_CR0,0,190,281_.jpg',
 'labels': 'SuperHero',
 'Genres': ['Action', 'Adventure', 'Fantasy'],
 'film_year': 2022,
 'chunk': 'Black Adam Nearly 5,000 years after he was bestowed with the almighty powers of the Egyptian gods - and imprisoned just as quickly - Black Adam is freed from his earthly tomb, ready to unleash his unique form of justice on the modern world.',
 'vector': array([ 0.00401279,  0.00175461, -0.00104652, ..., -0.00563261,
        -0.00292824,  0.0038763 ], dtype=float32)}

In [12]:
# Example PyMilvus utility API calls.

# # Count rows, incurs a call to .flush() first.
# This API call is not supported by Milvus Client.
# print(f"Count rows: {mc.num_entities(COLLECTION_NAME)}")

# View collection info, incurs a call to .flush() first.
start_time = time.time()
pprint.pprint(mc.describe_collection(COLLECTION_NAME))
end_time = time.time()
print(f"timing: {end_time - start_time} seconds")
print()

# Count rows without incurring call to .flush().
start_time = time.time()
res = mc.query( collection_name=COLLECTION_NAME, 
               filter="", 
               output_fields = ["count(*)"], )
pprint.pprint(res)
end_time = time.time()
print(f"timing: {end_time - start_time} seconds")

# # View rows without incurring call to .flush().
# OUTPUT_FIELDS = ['movie_index', 'title', 'description',
#   'poster_url', 'labels', 'Genres', 'film_year', 'chunk']
# res = mc.query( collection_name=COLLECTION_NAME, 
#                filter="id >= 0", 
#                output_fields = OUTPUT_FIELDS, )
# pprint.pprint(res)

{'aliases': [],
 'auto_id': True,
 'collection_id': 448076879578126663,
 'collection_name': 'imdb_metadata',
 'consistency_level': 3,
 'description': '',
 'enable_dynamic_field': True,
 'fields': [{'auto_id': True,
             'description': '',
             'field_id': 100,
             'is_primary': True,
             'name': 'id',
             'params': {},
             'type': <DataType.INT64: 5>},
            {'description': '',
             'field_id': 101,
             'name': 'vector',
             'params': {'dim': 1024},
             'type': <DataType.FLOAT_VECTOR: 101>}],
 'num_partitions': 1,
 'num_shards': 1,
 'properties': {}}
timing: 0.037996768951416016 seconds

[{'count(*)': 100}]
timing: 0.12284588813781738 seconds


In [13]:
# Define metadata fields you can filter on.
OUTPUT_FIELDS = list(df.columns)
# rename text to chunk
OUTPUT_FIELDS[OUTPUT_FIELDS.index('text')] = 'chunk'
OUTPUT_FIELDS

['movie_index',
 'title',
 'description',
 'poster_url',
 'labels',
 'Genres',
 'film_year',
 'chunk']

In [14]:
# List distinct genres
GENRES = list(set([genre for genres in df['Genres'] for genre in genres]))
GENRES

['Action',
 'Adventure',
 'Crime',
 'Horror',
 'Sci-Fi',
 'Thriller',
 'Drama',
 'Mystery',
 'Comedy',
 'Animation',
 'Fantasy']

In [15]:
if DATA_TYPE == "CSV":
    # Plot histogram of rating values.
    import matplotlib.pyplot as plt
    plt.figure(figsize=(4, 2))
    df['RatingValue'].hist();
    # Scale of 1-10.  Popular movies are rated >= 7.

## Ask a question about your data

So far in this demo notebook: 
1. Your custom data has been mapped into a vector embedding space
2. Those vector embeddings have been saved into a vector database

Next, you can ask a question about your custom data!

💡 In LLM vocabulary:
> **Query** is the generic term for user questions.  
A query is a list of multiple individual questions, up to maybe 1000 different questions!

> **Question** usually refers to a single user question.  
In our example below, the user question is "What is AUTOINDEX in Milvus Client?"

> **Semantic Search** = very fast search of the entire knowledge base to find the `TOP_K` documentation chunks with the closest embeddings to the user's query.

💡 The same model should always be used for consistency for all the embeddings data and the query.

In [16]:
# Define a sample question about your data.

# These 2 questions are for the CSV dataset.
QUESTION1 = "I'm a medical doctor of Lou Gehrig's disease."
QUESTION2 = "Bollywood"

# This question for the JSON dataset.
QUESTION3 = "Dystopia science fiction with a robot."

# Inspect the length of the query.
QUERY_LENGTH = len(QUESTION1)
print(f"query length: {QUERY_LENGTH}")

query length: 45


In [17]:
# SELECT A PARTICULAR QUESTION TO ASK.

SAMPLE_QUESTION = QUESTION3

## Execute a vector search

Search Milvus using [PyMilvus API](https://milvus.io/docs/search.md).

💡 By their nature, vector searches are "semantic" searches.  For example, if you were to search for "leaky faucet": 
> **Traditional Key-word Search** - either or both words "leaky", "faucet" would have to match some text in order to return a web page or link text to the document.

> **Semantic search** - results containing words "drippy" "taps" would be returned as well because these words mean the same thing even though they are different words.

In [18]:
def mc_run_search(question, filter_expression):
    # Embed the question using the same encoder.
    query_embeddings = _utils.embed_query(encoder, [question])
    TOP_K = 2

    # Return top k results with HNSW index.
    SEARCH_PARAMS = dict({
        # Re-use index param for num_candidate_nearest_neighbors.
        "ef": INDEX_PARAMS['efConstruction']
    })

    # Run semantic vector search using your query and the vector database.
    results = mc.search(
        COLLECTION_NAME,
        data=query_embeddings, 
        search_params=SEARCH_PARAMS,
        output_fields=OUTPUT_FIELDS, 
        # Milvus can utilize metadata in boolean expressions to filter search.
        filter=filter_expression,
        limit=TOP_K,
        consistency_level="Eventually"
    )

    # Assemble retrieved context and context metadata.
    # The search result is in the variable `results[0]`, which is type 
    # 'pymilvus.orm.search.SearchResult'. 
    METADATA_FIELDS = [f for f in OUTPUT_FIELDS if f != 'chunk']
    formatted_results, context, context_metadata = _utils.client_assemble_retrieved_context(
        results, metadata_fields=METADATA_FIELDS, num_shot_answers=TOP_K)
    
    return formatted_results, context, context_metadata

In [19]:
# Run the search.

# Metadata filters for CSV dataset.
# expression = "RatingValue >= 7"

# Metadata filters for JSON dataset.
# expression = 'film_year >= 2019'
expression = 'json_contains(Genres, "Sci-Fi") and film_year < 2019'
print(f"filter: {expression}")

start_time = time.time()
formatted_results, context, context_metadata = \
    mc_run_search(SAMPLE_QUESTION, expression)
elapsed_time = time.time() - start_time
print(f"Milvus Client search time for {len(chunk_list)} vectors: {elapsed_time} seconds")

# Inspect search result.
print(f"type: {type(formatted_results)}, count: {len(formatted_results)}")

filter: json_contains(Genres, "Sci-Fi") and film_year < 2019
Milvus Client search time for 100 vectors: 0.3123598098754883 seconds
type: <class 'list'>, count: 2


In [20]:
# Display poster link.
from IPython.display import Image
from IPython.display import display

# Loop through recommended movies, display poster, print metadata.
seen_movies = []
for i in range(len(context)):
    print(f"Retrieved result #{i+1}")
    print(f"distance = {formatted_results[i][0]}")
    # Get the movie_index
    movie_index = context_metadata[i]['movie_index']
    print(f"movie_index: {movie_index}")

    # Don't display the same movie_index twice.
    if movie_index in seen_movies:
        continue
    else:
        seen_movies.append(movie_index)
        # Display the first poster link as a rendered image
        if DATA_TYPE == "CSV":
            x = Image(url = context_metadata[i]['PosterLink'], width=150, height=200) 
        elif DATA_TYPE == "JSON":
            x = Image(url = context_metadata[i]['poster_url'], width=150, height=200) 
        display(x)

        # Print the rest of the movie info.
        pprint.pprint(f"Chunk text: {context[i]}")
        # print metadata except the movie_index and poster link.
        if DATA_TYPE == "CSV":
            for key, value in context_metadata[i].items():
                if ((key != 'PosterLink') or (key != 'movie_index')):
                    print(f"{key}: {value}")
            print()
        elif DATA_TYPE == "JSON":
            for key, value in context_metadata[i].items():
                if ((key != 'poster_url') or (key != 'movie_index')):
                    print(f"{key}: {value}")
            print()

Retrieved result #1
distance = 0.5802159309387207
movie_index: tt0434409


('Chunk text: V for Vendetta In a future British dystopian society, a shadowy '
 'freedom fighter, known only by the alias of "V", plots to overthrow the '
 'tyrannical government - with the help of a young woman.')
movie_index: tt0434409
title: V for Vendetta
description: In a future British dystopian society, a shadowy freedom fighter, known only by the alias of "V", plots to overthrow the tyrannical government - with the help of a young woman.
poster_url: https://m.media-amazon.com/images/M/MV5BOTI5ODc3NzExNV5BMl5BanBnXkFtZTcwNzYxNzQzMw@@._V1_QL75_UX190_CR0,0,190,281_.jpg
labels: SuperHero
Genres: ['Action', 'Drama', 'Sci-Fi']
film_year: 2005

Retrieved result #2
distance = 0.5398463606834412
movie_index: tt0489099


('Chunk text: Jumper A teenager with teleportation abilities suddenly finds '
 'himself in the middle of an ancient war between those like him and their '
 'sworn annihilators.')
movie_index: tt0489099
title: Jumper
description: A teenager with teleportation abilities suddenly finds himself in the middle of an ancient war between those like him and their sworn annihilators.
poster_url: https://m.media-amazon.com/images/M/MV5BMjEwOTkyOTI3M15BMl5BanBnXkFtZTcwNTQxMjU1MQ@@._V1_QL75_UX190_CR0,0,190,281_.jpg
labels: SuperHero
Genres: ['Action', 'Adventure', 'Sci-Fi']
film_year: 2008



In [21]:
# Drop collection
utility.drop_collection(COLLECTION_NAME)

In [22]:
# Props to Sebastian Raschka for this handy watermark.
# !pip install watermark

%load_ext watermark
%watermark -a 'Christy Bergman' -v -p torch,transformers,sentence_transformers,pymilvus,langchain --conda

Author: Christy Bergman

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

torch                : 2.2.1
transformers         : 4.39.1
sentence_transformers: 2.6.0
pymilvus             : 2.4.0
langchain            : 0.1.13

conda environment: py311

